# Final Project 

### Team members

Sai Swetha Vadrevu (YA34810)

Sai Kowshik Mayur Kaluvayi (IG25413)

In [2]:
# Importing necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import re
import nltk
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to C:\Users\SAI
[nltk_data]     SWETHA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Loading the datasets
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/y_train.csv')
X_final = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/X_final.csv')
y_final = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/y_final.csv')

In [4]:
# Downloading the necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Setting up the lemmatizer and the list of stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean a document
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Lowercase all text
    text = text.lower()

    # Split the text into words and remove stopwords, then lemmatize each word
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Join the words back into a single string
    text = ' '.join(words)
    return text

# Applying the cleaning function to every document in the training data
X_train['review'] = X_train['review'].apply(clean_text)

# Doing the same for the final dataset
X_final['review'] = X_final['review'].apply(clean_text)

[nltk_data] Downloading package stopwords to C:\Users\SAI
[nltk_data]     SWETHA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SAI
[nltk_data]     SWETHA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
X_train.head()

,review
0,shame swedish film swedish english subtitle fi...
1,know rather unfair comment movie without seein...
2,bread sharply skewer convention horror movie g...
3,reading ton good review movie decided take spi...
4,civil war wounded union soldier hide isolated ...


The data cleaning process involved several steps to prepare the text data for modeling. Here are the steps that were performed:

Downloading Necessary NLTK Data: The Natural Language Toolkit (NLTK) is a library in Python that provides tools for working with human language data. Here, we downloaded 'stopwords' and 'wordnet' from NLTK data. Stopwords are common words that do not contain important meaning and are usually removed from texts. Wordnet is a lexical database for the English language that helps the script determine the base word (lemma) of the inflected words.

Lemmatization and Stopwords Removal: Each review was split into individual words, and each word was lemmatized using the WordNet lemmatizer from NLTK. Lemmatization is the process of reducing a word to its base or root form (for example, "running" -> "run"). Stop words (commonly used words such as "the", "and", "a") were also removed from the reviews because they do not carry much meaningful information for the models.

Removing HTML tags: Any HTML tags present in the text were removed using a regular expression. These tags do not provide useful information for sentiment analysis.

Removing URLs: Any URLs present in the text were removed using a regular expression. URLs are typically not useful for sentiment analysis.

Removing Punctuation and Numbers: All punctuation marks and numbers were removed from the reviews. This is because they typically do not provide useful information for sentiment analysis. However, in some cases (like reviews with ratings), numbers might be important.

Lowercasing: All the text was converted to lower case. This is done to ensure that the same words in different cases are not considered as different words by the models.

After these steps, the cleaned data was ready to be vectorized and used to train the models.

First we have created a pipeline for logistic regression and trained the model.

In [4]:
# Creating a pipeline for Logistic Regression model
lr_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('lr', LogisticRegression()),
])

In [5]:
# Fit the pipeline on the training data
lr_pipeline.fit(X_train['review'], y_train['sentiment'])

C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('lr', LogisticRegression())])

In [7]:
# Predict the sentiment labels for the final set
y_pred_lr = lr_pipeline.predict(X_final['review'])

# Evaluate the performance of the model
accuracy_lr = accuracy_score(y_final['sentiment'], y_pred_lr)
classification_rep_lr = classification_report(y_final['sentiment'], y_pred_lr)

In [8]:
print("Accuracy score for Logistic Regression model is: ",accuracy_lr)
print(classification_rep_lr )

Accuracy score for Logistic Regression model is:  0.8844
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      5000
           1       0.88      0.89      0.89      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [9]:
# Performing cross-validation
lr_scores = cross_val_score(lr_pipeline, X_train['review'], y_train['sentiment'], cv=5)

C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

In [10]:
print("Logistic Regression model average accuracy: ", lr_scores.mean())

Logistic Regression model average accuracy:  0.88005


In [11]:
# Hyperparameter tuning for Logistic Regression model
params = {
    'vect__max_features': (None, 5000, 10000),
    'lr__C': (0.01, 0.1, 1, 10),
}

grid_search_lr = GridSearchCV(lr_pipeline, params, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train['review'], y_train['sentiment'])

print("Best score: ", grid_search_lr.best_score_)
print("Best parameters set: ")
best_parameters_lr = grid_search_lr.best_estimator_.get_params()
for param_name in sorted(params.keys()):
    print("\t", param_name, best_parameters_lr[param_name])

# Finally, evaluating the best model on the final dataset
best_model_lr = grid_search_lr.best_estimator_
predictions_lr = best_model_lr.predict(X_final['review'])
print("Final accuracy: ", accuracy_score(y_final['sentiment'], predictions_lr))

C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\SAI SWETHA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

Best score:  0.88645
Best parameters set: 
	 lr__C 0.1
	 vect__max_features None
Final accuracy:  0.8925


The accuracy score for logistic regression model is 88.44%. After performing hyper parameter tuning using greid search it increased to 89.25%.

Next we created a pipeline for random forest model and evaluated the model using pipeline.

In [14]:
# Creating a pipeline for Logistic Regression model
rf_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('rf', RandomForestClassifier())
])

In [15]:
# Fit the pipeline on the training data
rf_pipeline.fit(X_train['review'], y_train['sentiment'])

Pipeline(steps=[('vect', CountVectorizer()), ('rf', RandomForestClassifier())])

In [16]:
# Predict the sentiment labels for the final set
y_pred_rf = rf_pipeline.predict(X_final['review'])

# Evaluate the performance of the model
accuracy_rf = accuracy_score(y_final['sentiment'], y_pred_rf)
classification_rep_rf = classification_report(y_final['sentiment'], y_pred_rf)

In [17]:
print("Accuracy score for Random Forest model is: ",accuracy_rf)
print(classification_rep_rf)

Accuracy score for Random Forest model is:  0.8539
              precision    recall  f1-score   support

           0       0.86      0.85      0.85      5000
           1       0.85      0.86      0.85      5000

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [18]:
# Performing cross-validation for random forest
rf_scores = cross_val_score(rf_pipeline, X_train['review'], y_train['sentiment'], cv=5)

In [19]:
print("Random Forest model average accuracy: ", rf_scores.mean())

Random Forest model average accuracy:  0.8523250000000001


In [20]:
# Hyperparameter tuning for Random Forest model
# Define the parameters for the random search
params_rf = {
    'vect__max_features': [None, 5000, 10000],
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20],
}

# Use RandomizedSearchCV
random_search_rf = RandomizedSearchCV(rf_pipeline, params_rf, cv=5, scoring='accuracy', n_iter=10, random_state=42)
random_search_rf.fit(X_train['review'], y_train['sentiment'])

print("Best score: ", random_search_rf.best_score_)
print("Best parameters set: ")
best_parameters_rf = random_search_rf.best_estimator_.get_params()
for param_name in sorted(params_rf.keys()):
    print("\t", param_name, best_parameters_rf[param_name])

# Finally, evaluating the best model on the final dataset
best_model_rf = random_search_rf.best_estimator_
predictions_rf = best_model_rf.predict(X_final['review'])
print("Final accuracy: ", accuracy_score(y_final['sentiment'], predictions_rf))


Best score:  0.8526
Best parameters set: 
	 rf__max_depth None
	 rf__n_estimators 200
	 vect__max_features 10000
Final accuracy:  0.8532


The accuracy score for random forest model 85.39%. After performing hyper parameter tuning using randomised search the accuracy decreased to 85.32%.

We then created a pipeline for naive bayes classification model and evaluated the model.

In [21]:
# Creating a pipeline for naive bayes classification  model
nb_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [22]:
# Fit the pipeline on the training data
nb_pipeline.fit(X_train['review'], y_train['sentiment'])

Pipeline(steps=[('vect', CountVectorizer()), ('nb', MultinomialNB())])

In [23]:
# Predict the sentiment labels for the final set
y_pred_nb = nb_pipeline.predict(X_final['review'])

# Evaluate the performance of the model
accuracy_nb = accuracy_score(y_final['sentiment'], y_pred_nb)
classification_rep_nb = classification_report(y_final['sentiment'], y_pred_nb)

In [24]:
print("Accuracy score for Naive Bayes Classification model is: ",accuracy_nb)
print(classification_rep_nb)

Accuracy score for Naive Bayes Classification model is:  0.8649
              precision    recall  f1-score   support

           0       0.85      0.88      0.87      5000
           1       0.88      0.85      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000



In [25]:
# Hyperparameter tuning for Naive Bayes model
# Define the parameters for the random search
params_nb = {
    'vect__max_features': [None, 5000, 10000],
    'nb__alpha': [0.01, 0.1, 1, 10],
}

# Use RandomizedSearchCV
random_search_nb = RandomizedSearchCV(nb_pipeline, params_nb, cv=5, scoring='accuracy', n_iter=10, random_state=42)
random_search_nb.fit(X_train['review'], y_train['sentiment'])

print("Best score: ", random_search_nb.best_score_)
print("Best parameters set: ")
best_parameters_nb = random_search_nb.best_estimator_.get_params()
for param_name in sorted(params_nb.keys()):
    print("\t", param_name, best_parameters_nb[param_name])

# Finally, evaluating the best model on the final dataset
best_model_nb = random_search_nb.best_estimator_
predictions_nb = best_model_nb.predict(X_final['review'])
print("Final accuracy: ", accuracy_score(y_final['sentiment'], predictions_nb))


Best score:  0.851625
Best parameters set: 
	 nb__alpha 10
	 vect__max_features None
Final accuracy:  0.8645


The accuracy score for naive bayes classfication model 86.49%. After performing hyper parameter tuning using randomised search the accuracy decreased to 86.45%. 

### Conclusions
Logistic Regression: The logistic regression model performed the best among the three models with an accuracy score of 88.44%. After hyperparameter tuning with grid search, the performance increased to 89.25%. This indicates that logistic regression is a good fit for this data and the chosen hyperparameters improved the model's ability to generalize.

Random Forest: The random forest model achieved an accuracy score of 85.39%. However, after hyperparameter tuning with randomized search, the performance decreased slightly to 85.32%. This suggests that the random forest model may be overfitting the training data, and the chosen hyperparameters may not have been optimal.

Naive Bayes: The naive bayes model achieved an accuracy score of 86.49%. Like the random forest model, after hyperparameter tuning with randomized search, the performance decreased slightly to 86.45%. This indicates that the naive bayes model may also be overfitting the training data, and the chosen hyperparameters may not be optimal.

As our data set is balanced, accuracy is often a good metric to start with. Accuracy is the proportion of predictions that the model got right, i.e., the number of correct predictions divided by the total number of predictions. So we choose accuracy as an appropriate metric to evaluate models.

### Future Steps
Feature Engineering: To improve the performance of the models, we could look into additional feature engineering techniques. This could involve using TF-IDF vectorization instead of count vectorization, or creating additional features based on the text data, such as the length of reviews, the number of positive or negative words, etc.

Hyperparameter Tuning: The decrease in performance after hyperparameter tuning for the random forest and naive bayes models suggests that the chosen hyperparameters were not optimal. We could conduct more extensive hyperparameter tuning, perhaps using grid search instead of randomized search, to find better hyperparameters.

Deep Learning: Finally, if computational resources allow, we could explore deep learning models for this task. Recurrent Neural Networks (RNNs) and Transformers (like BERT) have shown excellent results in sentiment analysis and could potentially improve the performance of our models.